# Setup

In [1]:
!ls

Icerbeg or ship.ipynb  test.jpeg


In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
#import sys
#sys.path.insert(0,'/home/nidhin/fastai')

#export PYTHONPATH=$PYTHONPATH:/home/nidhin/fastai


In [4]:
from fastai.imports import *

In [5]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import pandas as pd
import numpy as np

In [6]:
!ls /home/nidhin/data/iceberg

models			  test.json	train	       valid
sample_submission.csv	  test.json.7z	train.json
sample_submission.csv.7z  tmp		train.json.7z


In [7]:
PATH = "/home/nidhin/data/iceberg/"

In [8]:
!ls {PATH}

models			  test.json	train	       valid
sample_submission.csv	  test.json.7z	train.json
sample_submission.csv.7z  tmp		train.json.7z


In [9]:
class_map = {0:'ice',1:'ship'}

In [10]:
train_df = pd.read_json(PATH+"/train.json")
train_df['classes'] = train_df['is_iceberg'].map(class_map)

test_df = pd.read_json(PATH+"/test.json")
test_df['classes'] = test_df['is_iceberg'].map(class_map)


ValueError: Could not reserve memory block

In [ ]:
train_df.head()

In [ ]:
def normalize_band(band):
    normalized_band = (band-np.min(band))/(np.max(band)-np.min(band))
    
    band = np.array(normalized_band).reshape((75, 75))
    pband = band - np.mean(band)
    pband[pband < band.std() * 2] = 0
    pband = np.mean(np.abs(np.gradient(pband)),axis=0)
    return pband
    
    #positive = band + np.amin(band)
    #normalized = positive / np.amax(positive)
    
    #return normalized
    
    #return band
    
    

    

def get_scaled_imgs(df):
    imgs = []
    for i, row in df.iterrows():
        band_1 = normalize_band(row['band_1'])
        band_2 = normalize_band(row['band_2'])
        
        band_3 = normalize_band(band_1 + band_2)

        imgs.append(np.dstack((band_1, band_2, band_3)))
    return np.array(imgs)



In [ ]:
from PIL import Image

In [ ]:
i = X_all[0]

In [ ]:
#img = Image.fromarray(i)
#img.save("test.png")

In [ ]:
X_all = get_scaled_imgs(train_df)
y_all = train_df['is_iceberg']
y_classes = train_df['classes']

In [ ]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [ ]:
print("Band 1")
plots(X_all[0:8,:,:,0],rows=2, titles=y_classes[0:8])

In [ ]:
print("Band 2")
plots(X_all[0:8,:,:,1],rows=2, titles=y_classes[0:8])

In [ ]:
print("Band 3")
plots(X_all[0:8,:,:,2],rows=2, titles=y_classes[0:8])

In [ ]:
train_df['is_iceberg'].value_counts()

In [ ]:
y_all = train_df['is_iceberg']

In [ ]:
#!rm -rf {PATH}/{train,valid}
#!mkdir -p {PATH}/{train,valid}/{ship,ice}


In [ ]:
!ls -alt {PATH}

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.20, random_state=42)

In [ ]:
class_map

In [ ]:
?os.makedirsf"{PATH}/train

In [ ]:
class_map.values()

In [ ]:
import shutil


In [ ]:
def save_images(X_train, X_test, y_train, y_test):
    
    for img_class in class_map.values():
        #shutil.rmtree(f"{PATH}train/{img_class}")
        os.makedirs(f"{PATH}train/{img_class}")
        
        #shutil.rmtree(f"{PATH}valid/{img_class}")
        os.makedirs(f"{PATH}valid/{img_class}")
    
    img_id =0
    
    
    # train 
    print ("Training")
    for img,y in zip(X_train,y_train):
        img_class = class_map[y]
        
        plt.imsave(f"{PATH}train/{img_class}/{img_id}.jpeg",img)
            
        
        img_id+=1
    
    # valid
    
    
    img_id =0
    print ("Validation")
    for img,y in zip(X_train,y_train):
        img_class = class_map[y]
        
        plt.imsave(f"{PATH}valid/{img_class}/{img_id}.jpeg",img)
        img_id+=1
        
def binary_loss(y, p):
    return np.mean(-(y * np.log(p) + (1-y)*np.log(1-p)))

In [ ]:
save_images(X_train, X_test, y_train, y_test)

In [ ]:
metrics = [f2]
learn = ConvLearner.pretrained(arch,data)

In [ ]:
arch=resnet34
sz=75

#data = ImageClassifierData.from_arrays(PATH, (X_train,y_train), val=(X_test,y_test),tfms=tfms_from_model(arch, sz))

#metrics = [f2]
data  = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch,sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 3)

In [ ]:
arch=resnet34
sz=125

data  = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch,sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 3)

In [ ]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),axis=0)
(metrics.log_loss(y,preds), accuracy(preds,y))

In [ ]:
preds

In [ ]:
??confusion_matrix

In [ ]:
preds.shape

In [ ]:
y.shape

In [ ]:
preds

In [ ]:
#log_preds,y = learn.TTA()
#probs = np.mean(np.exp(log_preds),0)
#preds = np.argmax(log_preds, axis=1)
#probs = np.exp(log_preds[:,1])

from sklearn.metrics import confusion_matrix
y_predicted= np.argmax(preds,axis=1)
cm = confusion_matrix(y, y_predicted)


plot_confusion_matrix(cm, data.classes)

In [ ]:
learn.predict_array(X_test)

In [ ]:
X_test